In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import splitfolders 
from tensorflow.keras.preprocessing.image import load_img 

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.applications.xception import Xception

In [3]:
def splitting(original_folder, splitted_folder, ratio):
  path1 = os.path.join(original_folder)
  path2 = os.path.join(splitted_folder)
  return splitfolders.ratio(path1, path2, seed = 0000, ratio = ratio, group_prefix=None)

In [4]:
#### input dataset that want to split
input_folder = 'forest-fire\\Dataset\\Dataset\\Training and Validation\\'  

output_folder= 'forest-fire\\Dataset\\Dataset\\train_val_split'

splitfolders.ratio(input_folder, output= output_folder, seed=1337, ratio = (0.7, 0.3))

Copying files: 1520 files [00:02, 601.95 files/s]


In [5]:
image_size = (224,224)
batch_size = 64

In [6]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=10.0,
    zoom_range=0.1,
    horizontal_flip=True,  
)

train_ds = train_gen.flow_from_directory(
    "forest-fire\\Dataset\\Dataset\\train_val_split\\train",
    seed=1,
    target_size=image_size,
    batch_size=batch_size,
)

Found 1064 images belonging to 2 classes.


In [7]:
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_ds = train_gen.flow_from_directory(
    "forest-fire\\Dataset\\Dataset\\Testing",
    seed=1,
    target_size=image_size,
    batch_size=batch_size,
)

Found 380 images belonging to 2 classes.


In [8]:
validation_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = validation_gen.flow_from_directory(
    "forest-fire\\Dataset\\Dataset\\train_val_split\\val",
    seed=1,
    target_size=image_size,
    batch_size=batch_size,
)

Found 456 images belonging to 2 classes.


In [9]:
def make_model(learning_rate, droprate):
    base_model = Xception(
        weights='imagenet',
        input_shape=(224, 224, 3),
        include_top=False
    )

    base_model.trainable = False

    inputs = keras.Input(shape=(224, 224, 3))

    base = base_model(inputs, training=False)
    vector = keras.layers..GlobalAveragePooling2D()(base)

    inner = keras.layers.Dense(100, activation='relu')(vector)
    drop = keras.layers.Dropout(droprate)(inner)

    outputs = keras.layers.Dense(2, activation='softmax')(drop)

    model = keras.Model(inputs, outputs)
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"],
    )
    
    return model

In [ ]:
model = tf.keras.Sequential([
    Xception(weights='imagenet', input_shape=(224, 224, 3), include_top=False),
    
])

In [10]:
model = make_model(learning_rate=0.0001, droprate=0.2)

checkpoint_filename = "checkpoint/xceptionv2_{epoch:02d}_{val_accuracy:.3f}.h5"

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filename,
        monitor="val_accuracy",
        save_best_only=True,
        mode='max'
    )
]

In [11]:
history = model.fit(train_ds, epochs=250, validation_data=val_ds, callbacks=callbacks)

Epoch 1/250
17/17 [==============================] - 22s 899ms/step - loss: 0.4905 - accuracy: 0.8271 - val_loss: 0.2832 - val_accuracy: 0.9693
Epoch 2/250
17/17 [==============================] - 10s 554ms/step - loss: 0.2091 - accuracy: 0.9831 - val_loss: 0.1512 - val_accuracy: 0.9781
Epoch 3/250
17/17 [==============================] - 10s 561ms/step - loss: 0.1215 - accuracy: 0.9878 - val_loss: 0.1018 - val_accuracy: 0.9825
Epoch 4/250
17/17 [==============================] - 11s 621ms/step - loss: 0.0880 - accuracy: 0.9859 - val_loss: 0.0794 - val_accuracy: 0.9825
Epoch 5/250
17/17 [==============================] - 11s 630ms/step - loss: 0.0673 - accuracy: 0.9878 - val_loss: 0.0674 - val_accuracy: 0.9825
Epoch 6/250
17/17 [==============================] - 11s 654ms/step - loss: 0.0545 - accuracy: 0.9897 - val_loss: 0.0604 - val_accuracy: 0.9803
Epoch 7/250
17/17 [==============================] - 11s 665ms/step - loss: 0.0521 - accuracy: 0.9897 - val_loss: 0.0551 - val_accuracy: